In [1]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

from src.config import FEATURES_DIR
import importlib

importlib.import_module("src/scripts/8_visualize_results.py", package=None)
from sklearn.metrics import roc_auc_score, average_precision_score

Results PF model

In [2]:
results_file = FEATURES_DIR + f"PF/cpkrr/results.csv"
results = pd.read_csv(results_file)
results.head()

,Unnamed: 0,patient,auc,aucpr,max_iter,fold
0,0,258,0.348653,0.548910,1,0
1,0,258,0.377876,0.559068,2,0
2,0,258,0.397143,0.569329,3,0
3,0,258,0.407596,0.573135,4,0
4,0,258,0.428461,0.594657,5,0


In [3]:
mean_results = results.loc[:,['auc', 'aucpr', 'max_iter', 'patient']].groupby(['patient', 'max_iter']).mean().groupby('max_iter').mean()
mean_results.head()

,auc,aucpr
max_iter,,
1,0.807299,0.755324
2,0.807104,0.755031
3,0.808816,0.757420
4,0.808475,0.757126
5,0.809032,0.756925


Get results of PI model

In [4]:
file = FEATURES_DIR + f"PI/results_cpkrr.parquet"
group_df = pd.read_parquet(FEATURES_DIR + "val_groups.parquet")
results_PI = get_predictions(file, group_df=group_df, group_name="group_id")
results_PI.head()

,prediction,annotation,group_id,index,Patient
0,-0.680558,-1,336,0,1543
1,-0.441825,-1,335,1,1543
2,-0.348977,-1,336,2,1543
3,-0.365684,-1,333,3,1543
4,-0.684190,-1,334,4,1543


In [5]:
# mean_results_PI = pd.DataFrame(columns=['Patient', 'auc', 'aucpr'])
auc = results_PI.groupby('Patient').apply(lambda x: roc_auc_score(x['annotation'], x['prediction']))
auc.name = 'auc'
aucpr = results_PI.groupby('Patient').apply(lambda x: average_precision_score(x['annotation'], x['prediction']))
aucpr.name = 'aucpr'
patient_results_PI = pd.concat([auc, aucpr], axis=1)
print(patient_results_PI)

              auc     aucpr
Patient                    
258      0.691582  0.619154
1543     0.713123  0.570155
5479     0.967055  0.786998
5943     0.603524  0.709320
6514     0.804068  0.327469
6811     0.514468  0.185239


In [6]:
mean_results_PI = patient_results_PI.mean()
print(mean_results_PI)

auc      0.715637
aucpr    0.533056
dtype: float64


In [7]:
mean_results.loc[0, :] = mean_results_PI
mean_results.sort_index(inplace=True)
mean_results.head()
mean_results.columns = ['AUROC-PF', 'AUPRC-PF']

Results Random PS model

In [8]:
results_random = pd.read_csv(FEATURES_DIR + f"PF/cpkrr/random_results.csv")
mean_results_random = results_random.loc[:,['auc', 'aucpr', 'max_iter', 'patient']].groupby(['patient', 'max_iter']).mean().groupby('max_iter').mean()
mean_results_random.columns = ['AUROC-Random', 'AUPRC-Random']
# mean_results_random = pd.melt(mean_results_random.reset_index(), id_vars=['max_iter'], value_vars=['PS-AUROC', 'PS-AUPRC'], var_name='metric', value_name='Value')

Make Plot

In [9]:
mean_results = pd.concat([mean_results, mean_results_random], axis=1)
mean_results['max_iter'] = mean_results.index
# mean_results['max_iter'].convert_dtypes = int
mean_results = pd.melt(mean_results, value_vars=['AUROC-PF', 'AUPRC-PF','AUROC-Random', 'AUPRC-Random'], id_vars='max_iter', var_name='metric', value_name='Value')
mean_results.head()

,max_iter,metric,Value
0,0,AUROC-PF,0.715637
1,1,AUROC-PF,0.807299
2,2,AUROC-PF,0.807104
3,3,AUROC-PF,0.808816
4,4,AUROC-PF,0.808475


In [13]:
import matplotlib
from matplotlib.ticker import MaxNLocator
matplotlib.use("pgf")
# %matplotlib inline
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

# matplotlib.use("qt5agg")
sns.set_palette("colorblind")
plt.close('all')
plt.figure(figsize=(6, 4))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
ax = sns.lineplot(mean_results, x='max_iter', y='Value', hue='metric', markers=['s','s','o','o'], style='metric', dashes=False, palette=[colors[0], colors[1], colors[0], colors[1]], markevery=1, lw=1.2, markersize=8)
# ax = sns.pointplot(mean_results_random, x='max_iter', y='Value', hue='metric', markers=['s','s','o', 'o'], scale=0.7)
# ax.legend(['AUROC', 'AUPRC'], loc='lower right')
# ax.plot([0], [mean_results_PI['auc']],  marker='*', markersize=7, color=colors[0], label='PI-AUROC', linestyle='None')
# ax.plot([0], [mean_results_PI['aucpr']],  marker='*', markersize=7, color=colors[1], label='PI-AUPRC', linestyle='None')
ax.legend(labels=None, loc='lower right', fontsize=14, fancybox=False, ncol=2, framealpha=1.)
ax.grid(True)
ax.set_xlabel('Iterations', fontsize=17)
ax.set_ylabel("Metric", fontsize=17)
ax.set_xlim(0, 35)
ax.set_ylim(0.3,0.85)
ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='minor', labelsize=14)
plt.tight_layout()
# ax.xaxis.set_major_locator(MaxNLocator(integer=True))
# plt.show()
plt.savefig('convergence_auc.pgf')

In [11]:
import os
os.getcwd()

'c:\\Users\\selinederooij\\surfdrive\\Code\\data_processing_library\\seizure_data_processing\\notebooks'